#CIS 5560 Term Project
Checking the python version

In [2]:
import sys
print('Python: {}'.format(sys.version))

Python: 3.5.2 (default, Nov 12 2018, 13:43:14) 
[GCC 5.4.0 20160609]

### Import Spark SQL and Spark ML Libraries

First, import the libraries you will need:

In [4]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

import numpy as np
from pyspark.sql import functions as F
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import LinearRegression
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.regression import GBTRegressor

### Load Source Data

1. After youth_risk_behaviours_surveillance.csv file is added to the data of the left frame, create a table using the UI, especially, "Upload File"
2. Click "Preview Table to view the table" and check the option of youth_risk_behaviours_surveillance.csv has a header as the first row: "First line is header"
3. Change the data type of the table columns if needed

The following should show your table name you created with inferSchmea option marked when you create it

In [7]:
#%fs ls /FileStore/tables/youth_risk_behaviours_surveillance.csv

### Prepare the Data
Select a subset of columns to use as *features* and label to predict. Here the prediction is done over 'Greater Risk data value'.

In [9]:
csv1 = spark.sql("SELECT * FROM youth_risk_behaviours_surveillance_csv")
csv1.createOrReplaceTempView("table1")
csv1.show(5)


+----+------------+--------------+----------+----------------+----------------+--------------------+---------------------+--------------------+-----------------+---------------+-----------------------+---------------------------------------+--------------------------------+---------------------------------+----------------------------------+--------------------+----------------------+--------------------------------------+-------------------------------+--------------------------------+---------------------------------+-----------+------+--------------------+-----+--------------------+-------+----------+------------+----------+--------+--------+--------+------------------+
YEAR|LocationAbbr| LocationDesc|DataSource| Topic| Subtopic| ShortQuestionText|Greater_Risk_Question| Description|Data_Value_Symbol|Data_Value_Type|Greater_Risk_Data_Value|Greater_Risk_Data_Value_Footnote_Symbol|Greater_Risk_Data_Value_Footnote|Greater_Risk_Low_Confidence_Limit|Greater_Risk_High_Confidence_Limit|Lesser_Risk_Question|Lesser_Risk_Data_Value|Lesser_Risk_Data_Value_Footnote_Symbol|Lesser_Risk_Data_Value_Footnote|Lesser_Risk_Low_Confidence_Limit|Lesser_Risk_High_Confidence_Limit|Sample_Size| Sex| Race|Grade| GeoLocation|TopicId|SubTopicID|QuestionCode|LocationId|StratID1|StratID2|StratID3|StratificationType|
+----+------------+--------------+----------+----------------+----------------+--------------------+---------------------+--------------------+-----------------+---------------+-----------------------+---------------------------------------+--------------------------------+---------------------------------+----------------------------------+--------------------+----------------------+--------------------------------------+-------------------------------+--------------------------------+---------------------------------+-----------+------+--------------------+-----+--------------------+-------+----------+------------+----------+--------+--------+--------+------------------+
2013| MEM| Memphis, TN| YRBSS|Sexual Behaviors|Sexual Behaviors| IUD or implant use| Did not use an IU...|to prevent pregna...| %| Percentage| null| null| null| null| null|Used an IUD (e.g....| null| null| null| null| null| 5| Total| Multiple Race| 9th|(35.149534, -90.0...| C04| C16| QNIUDIMP| 125| S1| R16| G2| Local|
1999| AS|American Samoa| YRBSS|Sexual Behaviors|Sexual Behaviors|No birth control use| Did not use any m...|among students wh...| %| Percentage| null| null| null| null| null| NULL| null| null| null| null| null| 3| Total| Multiple Race| 9th|(-31.653381, 161....| C04| C16| QNBCNONE| 60| S1| R16| G2| Territory|
1997| GU| Guam| YRBSS|Sexual Behaviors|Sexual Behaviors|No birth control use| Did not use any m...|among students wh...| %| Percentage| null| null| null| null| null| NULL| null| null| null| null| null| 0|Female|Black or African ...|Total|(13.444304, 144.7...| C04| C16| QNBCNONE| 66| S7| R12| G1| Territory|
2007| ME| Maine| YRBSS|Sexual Behaviors|Sexual Behaviors|Alcohol/drugs and...| Drank alcohol or ...|among students wh...| %| Percentage| null| null| null| null| null|Did not drink alc...| null| null| null| null| null| 0|Female| Multiple Race| 12th|(45.2542288940005...| C04| C16| H63| 23| S7| R16| G5| State|
2011| NJ| New Jersey| YRBSS|Sexual Behaviors| HIV Testing| HIV testing| Were never tested...|not counting test...| %| Percentage| 86.5079| null| null| 80.7077| 90.7638|Were ever tested ...| 13.4921| null| null| 9.2362| 19.2923| 146| Male| Hispanic or Latino|Total|(40.1305700480004...| C04| C17| H85| 34| S8| R13| G1| State|
+----+------------+--------------+----------+----------------+----------------+--------------------+---------------------+--------------------+-----------------+---------------+-----------------------+---------------------------------------+--------------------------------+---------------------------------+----------------------------------+--------------------+----------------------+--------------------------------------+----------

In [10]:
#show topics that can be used for prediction

df1 = spark.sql("SELECT count(*), Topic from table1 group by Topic")
df1.show(20,False)


+--------+---------------------------------------+
count(1)|Topic |
+--------+---------------------------------------+
801840 |Tobacco Use |
762240 |Sexual Behaviors |
315720 |Obesity, Overweight, and Weight Control|
668483 |Dietary Behaviors |
1176120 |Alcohol and Other Drug Use |
498120 |Physical Activity |
+--------+---------------------------------------+

# Data Cleaning

1. Filter the rows of topic that needs to be predicted
2. Drop the rows where value of label is null or ''.
3. Clip the values of columns of int, double type to where value is below 1 or above 99 percentile.

In [12]:
df2 = spark.sql("SELECT * from table1 where Topic = 'Obesity, Overweight, and Weight Control'")
df3 = df2.filter(df2.Greater_Risk_Data_Value.isNotNull())
df3.show(5, False)


+----+------------+------------+----------+---------------------------------------+----------------------+--------------------+---------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------+---------------+-----------------------+---------------------------------------+--------------------------------+---------------------------------+----------------------------------+--------------------------+----------------------+--------------------------------------+-------------------------------+--------------------------------+---------------------------------+-----------+------+------------------+-----+-----------------------------------------+-------+----------+------------+----------+--------+--------+--------+------------------+
YEAR|LocationAbbr|LocationDesc|DataSource|Topic |Subtopic |ShortQuestionText |Greater_Risk_Question |Description |Data_Value_Symbol|Data_Value_Type|Greater_Risk_Data_Value|Greater_Risk_Data_Value_Footnote_Symbol|Greater_Risk_Data_Value_Footnote|Greater_Risk_Low_Confidence_Limit|Greater_Risk_High_Confidence_Limit|Lesser_Risk_Question |Lesser_Risk_Data_Value|Lesser_Risk_Data_Value_Footnote_Symbol|Lesser_Risk_Data_Value_Footnote|Lesser_Risk_Low_Confidence_Limit|Lesser_Risk_High_Confidence_Limit|Sample_Size|Sex |Race |Grade|GeoLocation |TopicId|SubTopicID|QuestionCode|LocationId|StratID1|StratID2|StratID3|StratificationType|
+----+------------+------------+----------+---------------------------------------+----------------------+--------------------+---------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------+---------------+-----------------------+---------------------------------------+--------------------------------+---------------------------------+----------------------------------+--------------------------+----------------------+--------------------------------------+-------------------------------+--------------------------------+---------------------------------+-----------+------+------------------+-----+-----------------------------------------+-------+----------+------------+----------+--------+--------+--------+------------------+
2007|HI |Hawaii |YRBSS |Obesity, Overweight, and Weight Control|Obesity and Overweight|Obesity |Had obesity |students who were >= 95th percentile for body mass index, based on sex- and age-specific reference data from the 2000 CDC growth charts |% |Percentage |12.056 |null |null |8.9089 |16.1182 |NULL |87.944 |null |null |83.8818 |91.0911 |398 |Total |Asian |Total|(21.304850435000446, -157.85774940299973)|C07 |C26 |QNOBESE |15 |S1 |R11 |G1 |State |
1999|VT |Vermont |YRBSS |Obesity, Overweight, and Weight Control|Weight Control |Perception of weight|Described themselves as slightly or very overweight|NULL |% |Percentage |37.4113 |null |null |34.2417 |40.6927 |NULL |62.5887 |null |null |59.3073 |65.7583 |875 |Female|White |9th |(43.62538123900049, -72.51764079099962) |C07 |C27 |H68 |50 |S7 |R15 |G2 |State |
2009|MO |Missouri |YRBSS |Obesity, Overweight, and Weight Control|Obesity and Overweight|Obesity |Had obesity |students who were >= 95th percentile for body mass index, based on sex- and age-specific reference data from the 2000 CDC growth charts |% |Percentage |15.1423 |null |null |11.1293 |20.2721 |NULL |84.8577 |null |null |79.7279 |88.8707 |184 |Male |Total |9th |(38.635790776000476, -92.56630005299968) |C07 |C26 |QNOBESE |29 |S8 |R1 |G2 |State |
2017|CA |California |YRBSS |Obesity, Overweight, and Weight Control|Obesity and Overweight|Overweight |Were overweight |students who were >= 85th percentile but <95th percentile for body mass index, based on sex- and age-specific reference data from the 2000 CDC growth charts|% |Percentage |13.5197

### Select features and label

1. Select the relevant columns in a new dataframe and define the label.
2. Convert the string type columns into indexes using StringIndexer
3. Split the data in 70-30 train-test ratio.
4. Store the train-test data for both of the models separately.
5. Using Vector Assembler define the set of columns to be used as features.

In [14]:
data = df3.select("YEAR", "LocationDesc", "Sex", "Race", "Grade", "SubTopicID", "QuestionCode", "LocationId", "StratID1", col("Greater_Risk_Data_Value").alias("label"))
data.show(5, False)

data = StringIndexer(inputCol='LocationDesc', outputCol='LocationDesc'+"_index").fit(data).transform(data)
data = StringIndexer(inputCol='Sex', outputCol='Sex'+"_index").fit(data).transform(data)
data = StringIndexer(inputCol='Race', outputCol='Race'+"_index").fit(data).transform(data)
data = StringIndexer(inputCol='Grade', outputCol='Grade'+"_index").fit(data).transform(data)
data = StringIndexer(inputCol='SubTopicID', outputCol='SubTopicID'+"_index").fit(data).transform(data)
data = StringIndexer(inputCol='QuestionCode', outputCol='QuestionCode'+"_index").fit(data).transform(data)
data = StringIndexer(inputCol='LocationId', outputCol='LocationId'+"_index").fit(data).transform(data)
data = StringIndexer(inputCol='StratID1', outputCol='StratID1'+"_index").fit(data).transform(data)

# show indexed columns
data.show(5, False)

# Split the data
splits = data.randomSplit([0.7, 0.3])


# for linear regression
lr_train = splits[0]
lr_test = splits[1].withColumnRenamed("label", "trueLabel")


# for decision tree regression
dt_train = splits[0]
dt_test = splits[1].withColumnRenamed("label", "trueLabel")

+----+------------+------+------------------+-----+----------+------------+----------+--------+-------+
YEAR|LocationDesc|Sex |Race |Grade|SubTopicID|QuestionCode|LocationId|StratID1|label |
+----+------------+------+------------------+-----+----------+------------+----------+--------+-------+
2007|Hawaii |Total |Asian |Total|C26 |QNOBESE |15 |S1 |12.056 |
1999|Vermont |Female|White |9th |C27 |H68 |50 |S7 |37.4113|
2009|Missouri |Male |Total |9th |C26 |QNOBESE |29 |S8 |15.1423|
2017|California |Male |Asian |Total|C26 |QNOWT |6 |S8 |13.5197|
1997|Navajo |Female|Hispanic or Latino|Total|C27 |H69 |201 |S7 |43.718 |
+----+------------+------+------------------+-----+----------+------------+----------+--------+-------+
only showing top 5 rows

+----+------------+------+------------------+-----+----------+------------+----------+--------+-------+------------------+---------+----------+-----------+----------------+------------------+----------------+--------------+
YEAR|LocationDesc|Sex |Race |Grade|SubTopicID|QuestionCode|LocationId|StratID1|label |LocationDesc_index|Sex_index|Race_index|Grade_index|SubTopicID_index|QuestionCode_index|LocationId_index|StratID1_index|
+----+------------+------+------------------+-----+----------+------------+----------+--------+-------+------------------+---------+----------+-----------+----------------+------------------+----------------+--------------+
2007|Hawaii |Total |Asian |Total|C26 |QNOBESE |15 |S1 |12.056 |13.0 |0.0 |4.0 |0.0 |1.0 |3.0 |13.0 |0.0 |
1999|Vermont |Female|White |9th |C27 |H68 |50 |S7 |37.4113|5.0 |1.0 |1.0 |1.0 |0.0 |0.0 |5.0 |1.0 |
2009|Missouri |Male |Total |9th |C26 |QNOBESE |29 |S8 |15.1423|37.0 |2.0 |0.0 |1.0 |1.0 |3.0 |37.0 |2.0 |
2017|California |Male |Asian |Total|C26 |QNOWT |6 |S8 |13.5197|82.0 |2.0 |4.0 |0.0 |1.0 |2.0 |82.0 |2.0 |
1997|Navajo |Female|Hispanic or Latino|Total|C27 |H69 |201 |S7 |43.718 |59.0 |1.0 |2.0 |0.0 |0.0 |1.0 |59.0 |1.0 |
+----+------------+------+------------------+-----+----------+------------+----------+--------+-------+------------------+---------+----------+-----------+----------------+------------------+----------------+--------------+
only showing top 5 rows

In [15]:
assembler = VectorAssembler(inputCols = ["YEAR", "LocationDesc_index", "Sex_index", "Race_index", "Grade_index", "SubTopicID_index", "QuestionCode_index", "LocationId_index", "StratID1_index"], outputCol="features")

##Linear Regression Model

In [17]:
lr = LinearRegression(featuresCol = 'features', labelCol='label', maxIter=12345, regParam=0.4, elasticNetParam=0.7)
lr_pipeline = Pipeline(stages=[assembler, lr])

In [18]:
lr_model = lr_pipeline.fit(lr_train)
lr_prediction = lr_model.transform(lr_test)

Root Mean square error and Accuracy for linear regression

In [20]:
lr_evaluator = RegressionEvaluator(labelCol="trueLabel", predictionCol="prediction", metricName="rmse")

# Root Mean Square Error
rmse = lr_evaluator.evaluate(lr_prediction)
print("RMSE for linear regression is: %.4f" % rmse)

lr_accuracy = lr_evaluator.evaluate(lr_prediction, {lr_evaluator.metricName: "r2"})
print("linear regression model accuracy is: %.3f" %(lr_accuracy*100))


RMSE for linear regression is: 11.0122
linear regression model accuracy is: 66.059

Compare predicted values under 'prediction' column with actual values under 'trueLabel'

In [22]:
lr_predicted = lr_prediction.select("features", "prediction", "trueLabel")
lr_predicted.show()

+--------------------+------------------+---------+
 features| prediction|trueLabel|
+--------------------+------------------+---------+
[1991.0,24.0,1.0,...|47.076579588701115| 51.4703|
[1991.0,24.0,1.0,...| 37.46905508967197| 37.9117|
[1991.0,24.0,2.0,...| 48.15005640000112| 82.9905|
[1991.0,24.0,2.0,...| 38.26895820128685| 19.2196|
[1991.0,24.0,2.0,...| 38.26895820128685| 17.7994|
[1991.0,24.0,2.0,...| 47.94487612523728| 82.6863|
[1991.0,24.0,2.0,...|38.337351626208125| 24.6431|
[1991.0,24.0,2.0,...|38.337351626208125| 20.8407|
[1991.0,24.0,0.0,...| 36.73754540297837| 21.3051|
[1991.0,24.0,0.0,...| 36.73754540297837| 25.3406|
[1991.0,24.0,0.0,...|46.413463326928806| 66.9962|
[1991.0,24.0,0.0,...| 36.73754540297837| 17.954|
(9,[0,1,4,7],[199...| 36.53236512821453| 28.198|
[1991.0,24.0,0.0,...| 46.20828305216496| 65.9748|
[1991.0,24.0,0.0,...|36.600758553135805| 28.3183|
[1991.0,12.0,1.0,...| 47.28175986346496| 57.4587|
[1991.0,12.0,1.0,...|47.076579588701115| 42.5305|
[1991.0,12.0,1.0,...|47.076579588701115| 43.2768|
[1991.0,12.0,1.0,...| 37.46905508967197| 46.6537|
[1991.0,12.0,1.0,...| 37.46905508967197| 40.4063|
+--------------------+------------------+---------+
only showing top 20 rows

##Decision Tree Regression Model

In [24]:
dt = DecisionTreeRegressor(featuresCol='features', labelCol='label', maxBins=91)
dt_pipeline = Pipeline(stages=[assembler, dt])

Cross Validation

In [26]:
paramGrid = ParamGridBuilder().build()
cv = CrossValidator(estimator=dt_pipeline, evaluator=RegressionEvaluator(), estimatorParamMaps=paramGrid, numFolds=10)

dt_train.show()
dt_model = cv.fit(dt_train)

+----+------------+------+--------------------+-----+----------+------------+----------+--------+-------+------------------+---------+----------+-----------+----------------+------------------+----------------+--------------+
YEAR|LocationDesc| Sex| Race|Grade|SubTopicID|QuestionCode|LocationId|StratID1| label|LocationDesc_index|Sex_index|Race_index|Grade_index|SubTopicID_index|QuestionCode_index|LocationId_index|StratID1_index|
+----+------------+------+--------------------+-----+----------+------------+----------+--------+-------+------------------+---------+----------+-----------+----------------+------------------+----------------+--------------+
1991| Alabama|Female|Black or African ...|Total| C27| H68| 1| S7|26.3824| 24.0| 1.0| 3.0| 0.0| 0.0| 0.0| 24.0| 1.0|
1991| Alabama|Female|Black or African ...|Total| C27| H69| 1| S7|62.7505| 24.0| 1.0| 3.0| 0.0| 0.0| 1.0| 24.0| 1.0|
1991| Alabama|Female| Total| 11th| C27| H68| 1| S7|33.7372| 24.0| 1.0| 0.0| 3.0| 0.0| 0.0| 24.0| 1.0|
1991| Alabama|Female| Total| 12th| C27| H68| 1| S7|36.8454| 24.0| 1.0| 0.0| 4.0| 0.0| 0.0| 24.0| 1.0|
1991| Alabama|Female| Total| 12th| C27| H69| 1| S7|42.9037| 24.0| 1.0| 0.0| 4.0| 0.0| 1.0| 24.0| 1.0|
1991| Alabama|Female| Total| 9th| C27| H68| 1| S7|30.1276| 24.0| 1.0| 0.0| 1.0| 0.0| 0.0| 24.0| 1.0|
1991| Alabama|Female| White| 11th| C27| H68| 1| S7|37.3028| 24.0| 1.0| 1.0| 3.0| 0.0| 0.0| 24.0| 1.0|
1991| Alabama|Female| White| 12th| C27| H69| 1| S7|38.0129| 24.0| 1.0| 1.0| 4.0| 0.0| 1.0| 24.0| 1.0|
1991| Alabama|Female| White| 9th| C27| H69| 1| S7|43.7992| 24.0| 1.0| 1.0| 1.0| 0.0| 1.0| 24.0| 1.0|
1991| Alabama|Female| White|Total| C27| H68| 1| S7|35.6953| 24.0| 1.0| 1.0| 0.0| 0.0| 0.0| 24.0| 1.0|
1991| Alabama|Female| White|Total| C27| H69| 1| S7|41.7191| 24.0| 1.0| 1.0| 0.0| 0.0| 1.0| 24.0| 1.0|
1991| Alabama| Male| Total| 10th| C27| H68| 1| S8|23.1067| 24.0| 2.0| 0.0| 2.0| 0.0| 0.0| 24.0| 2.0|
1991| Alabama| Male| Total| 10th| C27| H69| 1| S8|78.2602| 24.0| 2.0| 0.0| 2.0| 0.0| 1.0| 24.0| 2.0|
1991| Alabama| Male| Total| 11th| C27| H69| 1| S8|81.9871| 24.0| 2.0| 0.0| 3.0| 0.0| 1.0| 24.0| 2.0|
1991| Alabama| Male| White| 10th| C27| H69| 1| S8|76.2583| 24.0| 2.0| 1.0| 2.0| 0.0| 1.0| 24.0| 2.0|
1991| Alabama| Male| White| 12th| C27| H68| 1| S8| 17.75| 24.0| 2.0| 1.0| 4.0| 0.0| 0.0| 24.0| 2.0|
1991| Alabama| Male| White| 9th| C27| H68| 1| S8|21.3704| 24.0| 2.0| 1.0| 1.0| 0.0| 0.0| 24.0| 2.0|
1991| Alabama| Male| White|Total| C27| H69| 1| S8|79.0282| 24.0| 2.0| 1.0| 0.0| 0.0| 1.0| 24.0| 2.0|
1991| Alabama| Total|Black or African ...| 10th| C27| H69| 1| S1|74.0024| 24.0| 0.0| 3.0| 2.0| 0.0| 1.0| 24.0| 0.0|
1991| Alabama| Total|Black or African ...|Total| C27| H68| 1| S1|20.7004| 24.0| 0.0| 3.0| 0.0| 0.0| 0.0| 24.0| 0.0|
+----+------------+------+--------------------+-----+----------+------------+----------+--------+-------+------------------+---------+----------+-----------+----------------+------------------+----------------+--------------+
only showing top 20 rows

Compare predicted values under 'prediction' column with actual values under 'trueLabel'

In [28]:
dt_prediction = dt_model.transform(dt_test)
dt_predicted = dt_prediction.select("features", "prediction", "trueLabel")
dt_predicted.show()

+--------------------+------------------+---------+
 features| prediction|trueLabel|
+--------------------+------------------+---------+
[1991.0,24.0,1.0,...|44.319894249439905| 51.4703|
[1991.0,24.0,1.0,...|32.015246747967474| 37.9117|
[1991.0,24.0,2.0,...| 76.33176680161941| 82.9905|
[1991.0,24.0,2.0,...|20.500030071289697| 19.2196|
[1991.0,24.0,2.0,...|20.500030071289697| 17.7994|
[1991.0,24.0,2.0,...| 76.33176680161941| 82.6863|
[1991.0,24.0,2.0,...|20.500030071289697| 24.6431|
[1991.0,24.0,2.0,...|20.500030071289697| 20.8407|
[1991.0,24.0,0.0,...|26.978359566281913| 21.3051|
[1991.0,24.0,0.0,...|26.978359566281913| 25.3406|
[1991.0,24.0,0.0,...| 66.54387727272726| 66.9962|
[1991.0,24.0,0.0,...|26.978359566281913| 17.954|
(9,[0,1,4,7],[199...|26.978359566281913| 28.198|
[1991.0,24.0,0.0,...|55.940884873069905| 65.9748|
[1991.0,24.0,0.0,...|26.978359566281913| 28.3183|
[1991.0,12.0,1.0,...| 51.99767753086417| 57.4587|
[1991.0,12.0,1.0,...|44.319894249439905| 42.5305|
[1991.0,12.0,1.0,...|44.319894249439905| 43.2768|
[1991.0,12.0,1.0,...|32.015246747967474| 46.6537|
[1991.0,12.0,1.0,...|32.015246747967474| 40.4063|
+--------------------+------------------+---------+
only showing top 20 rows

Root Mean square error and Accuracy for decision tree regression

In [30]:
dt_evaluator = RegressionEvaluator(labelCol="trueLabel", predictionCol="prediction", metricName="rmse")
dt_rmse = dt_evaluator.evaluate(dt_prediction)
print ("Root Mean Square Error (RMSE):", dt_rmse)


Root Mean Square Error (RMSE): 4.705244641890169

In [31]:
dt_accuracy_evaluator=RegressionEvaluator(labelCol="trueLabel", predictionCol="prediction",metricName="r2")
dt_accuracy=dt_accuracy_evaluator.evaluate(dt_prediction)
print("accuracy  on test data=%g" % (dt_accuracy*100))

accuracy on test data=93.8037